In [ ]:
#Import dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawings=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose




In [ ]:
#video capture from camera

In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame=cap.read()
    cv2.imshow("media pipeline",frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
#make detection


In [ ]:
#Determining Joints

In [8]:


cap=cv2.VideoCapture("gym3.mp4") 
counter=0
stage=None

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,) as pose:
        while cap.isOpened():
            ret,frame =cap.read()

            #recolor images to RGB
            image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            image_flags_writablee= False

            #Make detection
            result=pose.process(image)

            #color images back to BGR
            image_flags_writablee= True
            image=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)


            #Extract landmark
            try:
                landmarks=result.pose_landmarks.landmark
                
                #Get Coordinates
                shoulder=[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow=[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                #Calculate angles
                angle=calculate_angle(shoulder,elbow,wrist)
               
                
                #Visulaize angles
                cv2.putText(image,str(angle),
                            tuple(np.multiply(elbow,[1280,720]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
                           )
                
                #Curl counter Logic
                if angle > 160:
                    stage= "down"

                if angle < 50 and stage== "down" :
                    stage="up"
                    counter +=1
                    
            except:
                pass
           
            #render curl counter
            #setup status box
            
            cv2.rectangle(image,(0,0),(225,73),(255,255,255),-1)
            
            #rep data
            cv2.putText(image,"Reps",(15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA
                       )
            cv2.putText(image,str(counter),(10,60),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA
                       )
            #stage data
            cv2.putText(image,"Stage",(65,12),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA
                       )
            cv2.putText(image,stage,(60,60),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA
                       )
            

            #render detection
            mp_drawings.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                        mp_drawings.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                        mp_drawings.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


            cv2.imshow("media pipeline",image)
        
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
            
cap.release()
cv2.destroyAllWindows()

In [ ]:
#calculate joint angles

In [2]:
def calculate_angle(a,b,c):
    a=np.array(a)  #first
    b=np.array(b)  #mid
    c=np.array(c)  #end
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle=360.0-angle
        
    return angle 
    
    
    